In [7]:
#%run stdPackages.ipynb # this imports a lot of useful packages

%run/stdPackages.ipynb
#test2


UsageError: Line magic function `%run/stdPackages.ipynb` not found.


In [8]:
import base
import os 

def read_csv_with_path(directory_path, file_name):
    # Use os.path.join to create the full file path
    file_path = os.path.join(directory_path, file_name)
    
    # Use the read_csv function to load the data into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    return df

In [12]:
curr = os.getcwd()
HourlyLoad_df = read_csv_with_path(curr,"Total_load.csv")
HourlyLoad_df = HourlyLoad_df.dropna()
HourlyLoad_df['h_id'] = HourlyLoad_df['Time (CET/CEST)'].str[11:16]
Average_hourly_load = pd.Series(HourlyLoad_df.groupby('h_id')['Actual Total Load [MW] - BZN|DK2'].mean(), name = 'Average Hourly Load')

In [13]:
Average_hourly_load

h_id
00:00    1251.202740
01:00    1201.383562
02:00    1169.797260
03:00    1157.789041
04:00    1158.621918
05:00    1210.189041
06:00    1356.353425
07:00    1514.895890
08:00    1611.136986
09:00    1660.983562
10:00    1692.493151
11:00    1690.397260
12:00    1683.194521
13:00    1672.112329
14:00    1649.980822
15:00    1628.378082
16:00    1657.315068
17:00    1747.443836
18:00    1736.723288
19:00    1646.265753
20:00    1560.821918
21:00    1482.054795
22:00    1400.153425
23:00    1316.602740
Name: Average Hourly Load, dtype: float64